In [135]:
import tensorflow as tf

In [136]:
class RNNLayer(tf.Module):
    def __init__(self, out_features, name = None):
        super().__init__(name=name)
        self.is_built = False
        self.wr = tf.Variable(tf.random.normal([out_features, out_features]), name = 'wr')
        self.out_features = out_features

    def resetMemory(self):
        self.stores = tf.Variable(tf.zeros(self.stores.shape))
    
    def __call__(self, x):
        if not self.is_built:
            self.stores = tf.Variable(tf.zeros([x.shape[0], self.out_features]), name = 'stor', trainable=False)
            self.wb = tf.Variable(tf.random.normal([x.shape[-1], self.out_features]), name = 'w0')
            self.b = tf.Variable(tf.random.normal([self.out_features]), name = 'b')
            self.is_built = False

        self.stores = tf.matmul(x, self.wb) + tf.matmul(self.stores, self.wr) + self.b
        
        return tf.nn.relu(self.stores)

In [137]:
class Model(tf.Module):
    def __init__(self, Layers, name = None):
        super().__init__(name =  name)
        self.layers = Layers

    def __call__(self, x):
        return self.layers(x)

In [138]:
class LayerSeries(tf.Module):
    def __init__(self, layers, name = None):
        super().__init__(name = name)
        self.layers = layers
    def __call__(self, x):
        for lay in self.layers:
            x = lay(x)
        return x

In [139]:
class LayerBranch(tf.Module):
    def __init__(self, leftlayers, rightlayers, operation, name = None):
        super().__init__(name = name)
        self.leftlayers = leftlayers
        self.rightlayers = rightlayers
        self.opp = operation
    def __call__(self, x):
        xl = self.leftlayers(x)
        xr = self.rightlayers(x)
        return self.opp(xl, xr)